In [107]:
from packages import *
%matplotlib inline
import numpy as np, matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from sklearn.utils import shuffle, class_weight
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score
from collections import Counter
import h5py
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Advisor
The last step of the model, Advisor will predict stylistic preference from the 100-dimensional transformed activations

In [2]:
partition = load_pickle(os.path.join(LINK_DIR, 'partition_balanced.pkl'))
all_files = partition['train'] + partition['val'] + partition['test']
all_files.sort()
all_files = [os.path.join(IMG_DIR, f) for f in all_files]

In [3]:
likings = load_pickle(os.path.join(LINK_DIR, 'likings.pkl'))

In [6]:
activations = np.load(os.path.join(ACTIVATIONS_DIR, 'transformed_activations_cat.npy'), allow_pickle=False)

In [8]:
X, y = [], []
for i, f in enumerate(all_files):
    if f in likings:
        X.append(activations[i])
        y.append(likings[f])    

In [98]:
X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=.25)

In [99]:
Counter(likings.values())

Counter({0: 1026, 1: 380})

In [100]:
Counter(y_train)

Counter({0: 762, 1: 292})

In [102]:
sgd = SGDClassifier(n_jobs=5, validation_fraction=.05, class_weight='balanced', max_iter=1000)

In [103]:
sgd = sgd.fit(X_train, y_train)

In [104]:
predictions_sgd = sgd.predict(X_test)

In [105]:
accuracy_score(y_test, predictions_sgd)

0.4090909090909091

In [106]:
accuracy_score(y_train, sgd.predict(X_train))

0.4316888045540797

In [115]:
svc = LinearSVC(class_weight='balanced')

In [116]:
svc = svc.fit(X_train, y_train)

/home/kaan/anaconda3/envs/pennyworth_3.6/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [117]:
predictions_svc = svc.predict(X_test)

In [119]:
accuracy_score(y_test, predictions_svc)

0.7642045454545454

In [118]:
f1_score(y_test, predictions_svc)

0.3252032520325203

In [120]:
f1_score(y_train, sgd.predict(X_train))

0.47038019451812557